## ExtractTablesfromPDF

In this notebook, run the Table Transformer - which is actually a [DETR](https://arxiv.org/abs/2005.12872) model - by Microsoft Research to perform table detection and on documents (Pdfs and Images).


In [1]:
import time

start_time = time.time()

In [2]:
!pip install -q git+https://github.com/huggingface/transformers.GIT

  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.GIT 'C:\Users\engin\AppData\Local\Temp\pip-req-build-hyyq28ew' did not run successfully.
  │ exit code: 128
  ╰─> [2 lines of output]
      remote: Repository not found.
      fatal: repository 'https://github.com/huggingface/transformers.GIT/' not found
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.GIT 'C:\Users\engin\AppData\Local\Temp\pip-req-build-hyyq28ew' did not run successfully.
│ exit code: 128
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install -q timm
!pip install matplotlib
!pip install pdf2image


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached pdf2image-1.17.0-py3-none-any.whl.metadata (6.2 kB)
Using cached pdf2image-1.17.0-py3-none-any.whl (11 kB)


In [ ]:
# !pip uninstall tokenizers


In [ ]:
!pip install tokenizers==0.14.0


## Table detection

Table detection is the task of detect tables in document images, like PDFs.

Let's load a PDF and see how our model does.


## Convert the pdf_Pages to images:


In [8]:
# from pdf2image import convert_from_path
# import os

# def extract_images_from_pdf(pdf_path, output_folder):
#     """
#     Extracts all pages from a PDF file and saves them as images
#     in the specified output folder.

#     Args:
#     pdf_path (str): The path to the PDF file.
#     output_folder (str): The folder where the extracted images will be saved.
#     """

#     # Create the output folder if it does not exist
#     if not os.path.exists(output_folder):
#         os.makedirs(output_folder)

#     # Extract the base filename from the PDF path
#     pdf_filename = os.path.basename(pdf_path).replace(".pdf", "")

#     # Convert PDF pages to images
#     images = convert_from_path(pdf_path)

#     # Save each page as an image
#     for i, image in enumerate(images):
#         image.save(os.path.join(output_folder, f"{pdf_filename}_page_{i}.png"), "PNG")

#     return images


# # Example usage
# pdf_path = r"C:\A\00Master\Demo\data acc\DIN EN 206.pdf"
# output_folder = r"C:\A\00Master\Demo\data acc\testtest"
# imgs = extract_images_from_pdf(pdf_path, output_folder)

KeyboardInterrupt: 

time: 35.3 s (started: 2023-12-31 09:49:08 +01:00)


## Feature Extractor

- Appling the regular image preprocessing using `DetrImageProcessor`. The feature extractor will resize the image (minimum size = 800, max size = 1333), and normalize it across the channels using the ImageNet mean and standard deviation.


In [ ]:
%reload_ext autotime
from transformers import DetrImageProcessor,TableTransformerForObjectDetection
from transformers import TableTransformerForObjectDetection



# feature_extractor = DetrFeatureExtractor()    => # it will be deleted in the future 

feature_extractor = DetrImageProcessor()
encoding = feature_extractor(imgs, return_tensors="pt")
# encoding = feature_extractor(image, return_tensors="pt")


model = TableTransformerForObjectDetection.from_pretrained("microsoft/table-transformer-detection")

import torch

with torch.no_grad():
  outputs = model(**encoding)



In [9]:
import os
def detect_and_crop_tables(images, model, feature_extractor, output_folder, pdf_filename, padding=0):
    from PIL import Image
    import torch

    def crop_tables(image, page_num):
        encoding = feature_extractor(image, return_tensors="pt")
        with torch.no_grad():
            outputs = model(**encoding)

        width, height = image.size
        target_sizes = [(height, width)]
        results = feature_extractor.post_process_object_detection(outputs, threshold=0.9, target_sizes=target_sizes)[0]

        high_score_tables = [(score, label, box) for score, label, box in
                             zip(results['scores'].tolist(),
                                  results['labels'].tolist(), 
                                  results['boxes'].tolist())
                             if score > 0.9]

        if high_score_tables:
            for idx, (_, _, (xmin, ymin, xmax, ymax)) in enumerate(high_score_tables):
                xmin -= padding
                ymin -= padding
                xmax += padding
                ymax += padding
                table_img = image.crop((xmin, ymin, xmax, ymax))
                table_img.save(os.path.join(output_folder, f"{pdf_filename}_page_{page_num}_table_{idx}.png"), "PNG")

    for i, image in enumerate(images):
        crop_tables(image, i)


time: 0 ns (started: 2023-12-31 09:50:27 +01:00)


In [21]:
# %reload_ext autotime
from PIL import Image

file_path = r"C:\A\00Master\Demo\data acc\testtest\DIN EN 206_page_87.png"
image = Image.open(file_path).convert("RGB")
width, height = image.size
image.resize((int(width*0.5), int(height*0.5)))

from transformers import DetrImageProcessor,TableTransformerForObjectDetection
from transformers import TableTransformerForObjectDetection



# feature_extractor = DetrFeatureExtractor()    => # it will be deleted in the future 

feature_extractor = DetrImageProcessor()
encoding = feature_extractor(image, return_tensors="pt")
# encoding = feature_extractor(image, return_tensors="pt")


model = TableTransformerForObjectDetection.from_pretrained("microsoft/table-transformer-detection")

import torch

with torch.no_grad():
  outputs = model(**encoding)


# rescale bounding boxes
width, height = image.size
target_sizes = [(height, width)] * len(outputs.logits)  # Make sure to pass the correct number of target sizes

results = feature_extractor.post_process_object_detection(outputs,
                                                           threshold=0.9, 
                                                           target_sizes=target_sizes)[0]

print(results)
print(type(results))

Some weights of the model checkpoint at microsoft/table-transformer-detection were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'scores': tensor([0.9997]), 'labels': tensor([0]), 'boxes': tensor([[ 118.8181, 1222.5436, 1445.8184, 2016.4642]])}
<class 'dict'>
time: 1.5 s (started: 2023-12-31 10:12:38 +01:00)


In [23]:
import pytesseract
from PIL import Image, ImageDraw
from transformers import DetrImageProcessor, TableTransformerForObjectDetection
import torch

# Set the path to the Tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Load the image
file_path = r'C:\A\00Master\Demo\data acc\testtest\DIN EN 206_page_69.png'
image = Image.open(file_path).convert("RGB")

# Detect tables
feature_extractor = DetrImageProcessor()
model = TableTransformerForObjectDetection.from_pretrained("microsoft/table-transformer-detection")

encoding = feature_extractor(image, return_tensors="pt")
with torch.no_grad():
    outputs = model(**encoding)

# Rescale bounding boxes
width, height = image.size
target_sizes = [(height, width)] * len(outputs.logits)
results = feature_extractor.post_process_object_detection(outputs, threshold=0.9, target_sizes=target_sizes)[0]

# Create a copy of the image to perform OCR
image_for_ocr = image.copy()
draw = ImageDraw.Draw(image_for_ocr)

# Cover detected tables with white rectangles
for box in results['boxes']:
    draw.rectangle([(box[0], box[1]), (box[2], box[3])], fill="white")

# Perform OCR on the image with tables excluded
text_without_tables = pytesseract.image_to_string(image_for_ocr, lang='deu')

print(text_without_tables)
def write_text_to_file(text, file_path):
    """
    Writes given text to a .txt file at the specified file path.

    :param text: The text to be written to the file.
    :param file_path: The path (including filename) where the file should be saved.
    """
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(text)

# Example usage

output_file_path = 'outputSHaddadwithgermanwithoutonetable.txt'  # Specify your desired file path

write_text_to_file(text_without_tables, output_file_path)

Some weights of the model checkpoint at microsoft/table-transformer-detection were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


10/06/2023 08:33:34

Printed copies are uncontrolled

DIN EN 206:2021-06
EN 206:2013+A2:2021 (D)

ANMERKUNG Weitere Informationen enthält Anhang L, Zeile 20.

(4) Für Leichtbeton, der mit ungesättigter Gesteinskörnung dosiert wird, muss die Dauer zwischen dem
Erstmischen und dem Ende des letzten Mischvorgangs (z.B. erneutes Mischen in einem Fahrmischer) so
verlängert werden, dass die Wasseraufnahme der Gesteinskörnung und das nachfolgende Entweichen der
Luft aus der leichten Gesteinskörnung keine nachteilige Auswirkung auf die Eigenschaften des Festbetons
haben.

(5) Die Zusammensetzung des Frischbetons darf nach Verlassen des Mischers nicht verändert werden.

9.9 Verfahren der Produktionskontrolle

(1) Ausgangsstoffe, Ausstattung, Herstellverfahren und Beton müssen in Hinblick auf ihre Konformität mit
den Festlegungen für den Beton und den Anforderungen dieser Norm überwacht werden. Die
Produktionskontrolle muss so angelegt sein, dass wesentliche Änderungen, die die Eigenschaften
beei

In [12]:
# import pytesseract
# from PIL import Image
# import pandas as pd
# from transformers import DetrImageProcessor, TableTransformerForObjectDetection
# import torch
# import json

# # Configure PyTesseract
# pytesseract.pytesseract.tesseract_cmd = r'path_to_tesseract.exe'  # Update the path

# # Load image
# file_path = r"your_image_path.png"
# image = Image.open(file_path).convert("RGB")

# # Initialize the feature extractor and model
# feature_extractor = DetrImageProcessor()
# model = TableTransformerForObjectDetection.from_pretrained("microsoft/table-transformer-detection")

# # Process image for table detection
# encoding = feature_extractor(image, return_tensors="pt")
# with torch.no_grad():
#     outputs = model(**encoding)

# # Rescale bounding boxes
# width, height = image.size
# target_sizes = [(height, width)] * len(outputs.logits)
# results = feature_extractor.post_process_object_detection(outputs, threshold=0.9, target_sizes=target_sizes)[0]

# # Extract tables and texts
# data_json = {"tables": [], "texts": []}
# for box in results['boxes']:
#     # Crop table from image
#     cropped_img = image.crop((box[0], box[1], box[2], box[3]))
#     # TODO: Extract table data from cropped_img and append to data_json["tables"]

# # Extract text outside tables
# # TODO: Define areas outside tables and use pytesseract to extract text
# # Extracted text can be appended to data_json["texts"]

# # Save to JSON file
# with open('extracted_data.json', 'w') as outfile:
#     json.dump(data_json, outfile)


{'scores': tensor([0.9943, 0.9992]), 'labels': tensor([0, 0]), 'boxes': tensor([[ 383.1478,  696.4118, 1154.8029, 1158.4928],
        [ 518.7551, 1757.9453, 1108.2152, 2017.1786]])}
time: 16 ms (started: 2023-12-31 09:53:15 +01:00)


## Crop PDF images

- based on the results generated by the model to extract images that contain tables with a high probability of 0.9.


### with adding Padding of 20PXL to the Images


In [ ]:

%reload_ext autotime

def crop_tables(image, page_num):
    encoding = feature_extractor(image, return_tensors="pt")

    with torch.no_grad():
        outputs = model(**encoding)

    width, height = image.size
    target_sizes = [(height, width)]
    results = feature_extractor.post_process_object_detection(outputs, threshold=0.9, target_sizes=target_sizes)[0]

    # Crop tables with scores above 90
    high_score_tables = [(score, label, box) for score, label, box in
                         zip(results['scores'].tolist(), results['labels'].tolist(), results['boxes'].tolist())
                         if score > 0.9]

    if high_score_tables:
        for idx, (_, _, (xmin, ymin, xmax, ymax)) in enumerate(high_score_tables):
            # Increase crop area
            xmin -= 20  
            ymin -= 20
            xmax += 20
            ymax += 20
            
            # Crop the table using PIL with adjusted coordinates
            table_img = image.crop((xmin, ymin, xmax, ymax))
            # Save the cropped table image
            table_img.save(os.path.join(output_folder, f"{pdf_filename}_page_{page_num}_table_{idx}.png"), "PNG")

# Process each page in the PDF
for i, image in enumerate(images):
    crop_tables(image, i)



## Deleting Images

- Removing images that do not contain tables.


In [ ]:
import os

# Folder path
folder_path = r"ExtractTableandCropImages"
# Specific word in the file name
specific_word = "table"

# List all files in the folder
files = os.listdir(folder_path)

# Iterate through the files and delete those that do not have the specific word
for file in files:
    if specific_word not in file:
        file_path = os.path.join(folder_path, file)
        os.remove(file_path)
        print(f"Deleted: {file}")

print("Deletion process completed.")

### Time


In [ ]:
end_time = time.time()
execution_time = end_time - start_time
execution_time_in_minutes = execution_time / 60
print("Execution time :", execution_time, "seconds")
print("Execution time:", execution_time_in_minutes, "Minutes")